In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/github_trending_repos.csv")

df.head()

,owner,repo_name,full_name,description,language,stars,forks,stars_period,contributors_count,url,search_language,timeframe,scraped_at
0,sansan0,TrendRadar,sansan0/TrendRadar,🎯 告别信息过载，AI 助你看懂新闻资讯热点，简单的舆情监控分析 - 多平台热点聚合+基于 ...,Python,36604,19908,1067,0,https://github.com/sansan0/TrendRadar,overall,daily,2025-12-03 17:34:14.528785
1,google,adk-go,google/adk-go,"An open-source, code-first Go toolkit for buil...",Go,5920,396,118,0,https://github.com/google/adk-go,overall,daily,2025-12-03 17:34:14.529175
2,TapXWorld,ChinaTextbook,TapXWorld/ChinaTextbook,所有小初高、大学PDF教材。,Roff,60750,13494,226,0,https://github.com/TapXWorld/ChinaTextbook,overall,daily,2025-12-03 17:34:14.529535
3,yeongpin,cursor-free-vip,yeongpin/cursor-free-vip,[Support 0.49.x]（Reset Cursor AI MachineID & B...,Python,44916,5365,250,0,https://github.com/yeongpin/cursor-free-vip,overall,daily,2025-12-03 17:34:14.529894
4,nvm-sh,nvm,nvm-sh/nvm,Node Version Manager - POSIX-compliant bash sc...,Shell,90205,9641,67,0,https://github.com/nvm-sh/nvm,overall,daily,2025-12-03 17:34:14.530228


In [ ]:
target = 'stars_period'
y = df[target]

relevant_features = ['stars', 'forks', 'contributors_count', 'search_language', 'timeframe', 'language']
dropped_features = ['owner', 'repo_name', 'description', 'url', 'scraped_at', 'stars_period']

0    1067
1     118
2     226
3     250
4      67
Name: stars_period, dtype: int64